# AWS-ASDI
EDA on the NOAA Global Historical Climatology Network Daily Dataset. This dataset contains over 200 years worth of climate data, and we will analyze the dataset through the use of data science tools that are scalable (Modin, NumS, and Ray). Because the tools we use are scalable, we are able to run the same code on a laptop and cluster of nodes. Before getting started, make sure you set `num_cpus` in `ray_init()` to the number of physical cores on the CPU running this notebook for optimal performance, otherwise Ray might automatically set it to logical cores that include hyperthreading. Additionally, for cluster setup, the only thing that needs to be changed is properly setting the correct parameters in `ray.init()` and now the same notebook should be able to scale.

Confirm everything is installed through:
```sh
pip3 install -r requirements
```

When done with the notebook, you can properly shutdown Ray to free all processes and memory using:
```python
ray.shutdown()
```
For convenience, this cell is included at the end of the notebook.

In [1]:
local = True #Change this flag to False to download data off of s3 storage directly
n_jobs = 32 #this should be set to the number of logical cores

In [2]:
import ray
ray.init(ignore_reinit_error=True, num_cpus=n_jobs, _temp_dir="/home/brian/external/aws-asdi/ray_temp"); # Remove or reassign _temp_dir if not enough disk storage on device
#ray.init(ignore_reinit_error=True, address="auto") #ray.init() config for cluster setup
import modin.pandas as pd
import pandas
from nums import numpy as nps
from nums.core import settings
from nums.experimental import nums_modin
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import plotly.express as px
import plotly.graph_objects as go
import os
import warnings
warnings.filterwarnings("ignore")

2021-08-12 01:59:16,965	INFO services.py:1274 -- View the Ray dashboard at http://127.0.0.1:8265


Setting for NumS to assign head and worker nodes.

In [3]:
#settings.cluster_shape = (len(ray.nodes())-1, 1)

## Downloading the Data and File Organization
* [Data](https://registry.opendata.aws/noaa-ghcn/)
* [Documentation](https://docs.opendata.aws/noaa-ghcn-pds/readme.html)

We can use AWS CLI to speed things up by downloading it locally. More info about installation is provided [here](https://aws.amazon.com/cli/).

To view files in terminal/command line:
```sh
aws s3 ls s3://noaa-ghcn-pds/ --no-sign-request
```

To download locally (*about 100GB of free data is needed on disk*), we can download directly to our `data/` directory:
```sh
mkdir data
aws s3 cp --recursive s3://noaa-ghcn-pds/csv/ --no-sign-request data
```

If downloading will take too much storage space, we can run this notebook completely off of memory, by setting the `local` tag to `False` in the cell directly above. Some `.txt` files will be directly loaded from S3, as they are relatively small and are just lookup tables/directories for the `.csv` files.

File directory should look like this:
```
├── data/
├── figures/
├── eda.ipynb
├── README.md
└── requirements.txt
```

Here, we test downloading a `.csv` file directly from the AWS S3 bucket to memory. Ensure this is working before running the next cells. This notebook will do all of the data cleaning of DataFrames in memory. Because the S3 bucket is public, there should be no need to insert any AWS/S3 credentials.

In [4]:
%%time
climate_2020 = pd.read_csv('s3://noaa-ghcn-pds/csv/2020.csv', header=None)
climate_2020.columns = ["ID", "YEAR/MONTH/DAY", "ELEMENT", "DATA VALUE", "M-FLAG", "Q-FLAG", "S-FLAG", "OBS-TIME"]
climate_2020["YEAR/MONTH/DAY"] = pd.to_datetime(climate_2020["YEAR/MONTH/DAY"], format="%Y%m%d")
del climate_2020 # delete from memory

CPU times: user 5.55 s, sys: 3.57 s, total: 9.12 s
Wall time: 39.1 s


In [5]:
%%time
climate_2020 = pandas.read_csv('s3://noaa-ghcn-pds/csv/2020.csv', header=None)
climate_2020.columns = ["ID", "YEAR/MONTH/DAY", "ELEMENT", "DATA VALUE", "M-FLAG", "Q-FLAG", "S-FLAG", "OBS-TIME"]
climate_2020["YEAR/MONTH/DAY"] = pd.to_datetime(climate_2020["YEAR/MONTH/DAY"], format="%Y%m%d")
del climate_2020 # delete from memory

CPU times: user 23.5 s, sys: 7.53 s, total: 31 s
Wall time: 58.7 s


The times below are tested on a single node setup. As we can see, the speedups are not reliable as it is dependent on our connection speed between your network to the S3 bucket (located on us-east-1):

**modin**
```
CPU times: user 11.6 s, sys: 2.99 s, total: 14.6 s
Wall time: 1min 42s
```
**pandas**
```
CPU times: user 25.3 s, sys: 3.68 s, total: 29 s
Wall time: 1min 22s
```

In [6]:
%%time
#skip if not downloaded locally
climate_2020 = pd.read_csv('data/2020.csv', header=None)
climate_2020.columns = ["ID", "YEAR/MONTH/DAY", "ELEMENT", "DATA VALUE", "M-FLAG", "Q-FLAG", "S-FLAG", "OBS-TIME"]
climate_2020["YEAR/MONTH/DAY"] = pd.to_datetime(climate_2020["YEAR/MONTH/DAY"], format="%Y%m%d")
del climate_2020 # delete from memory

CPU times: user 1.85 s, sys: 1.01 s, total: 2.86 s
Wall time: 10.5 s


In [7]:
%%time
#skip if not downloaded locally
climate_2020 = pandas.read_csv('data/2020.csv', header=None)
climate_2020.columns = ["ID", "YEAR/MONTH/DAY", "ELEMENT", "DATA VALUE", "M-FLAG", "Q-FLAG", "S-FLAG", "OBS-TIME"]
climate_2020["YEAR/MONTH/DAY"] = pd.to_datetime(climate_2020["YEAR/MONTH/DAY"], format="%Y%m%d")
del climate_2020 # delete from memory

CPU times: user 17.5 s, sys: 3.57 s, total: 21.1 s
Wall time: 20.4 s


We can try the same cell again, but with data already stored to the local filesystem for a more reliable benchmark. As we can see, a significant speedup for a file that is 1 GB in size:

**modin**
```
CPU times: user 1.3 s, sys: 518 ms, total: 1.82 s
Wall time: 2.92 s
```
**pandas**
```
CPU times: user 17 s, sys: 2.16 s, total: 19.2 s
Wall time: 18.7 s
```

## Global Variables and DataFrames
Some variables and DataFrames that will be useful in parsing out data.

For inventory, data is stored like this:

```
Variable	Columns	Type
ID	1-11	CHARACTER
LATITUDE	13-20	REAL
LONGITUDE	22-30	REAL
ELEMENT	32-35	CHARACTER
FIRSTYEAR	37-40	INTEGER
LASTYEAR	42-45	INTEGER
```

Sample query
```
ACW00011604  17.1167  -61.7833 TMAX 1949 1949
```

In [8]:
inventory = pd.read_fwf('s3://noaa-ghcn-pds/ghcnd-inventory.txt', widths=[12, 9, 10, 4, 5, 5], header=None)
inventory.columns = ["ID", "LATITUDE", "LONGITUDE", "ELEMENT", "FIRSTYEAR", "LASTYEAR"]
inventory

,ID,LATITUDE,LONGITUDE,ELEMENT,FIRSTYEAR,LASTYEAR
0,ACW00011604,17.1167,-61.7833,TMAX,1949,1949
1,ACW00011604,17.1167,-61.7833,TMIN,1949,1949
2,ACW00011604,17.1167,-61.7833,PRCP,1949,1949
3,ACW00011604,17.1167,-61.7833,SNOW,1949,1949
4,ACW00011604,17.1167,-61.7833,SNWD,1949,1949
...,...,...,...,...,...,...
704607,ZI000067983,-20.2000,32.6160,PRCP,1951,2020
704608,ZI000067983,-20.2000,32.6160,TAVG,1962,2020
704609,ZI000067991,-22.2170,30.0000,TMAX,1951,1990
704610,ZI000067991,-22.2170,30.0000,TMIN,1951,1990


For stations, the data is formatted like this:
```
Variable	Columns	Type	Example
ID	1-11	Character	EI000003980
LATITUDE	13-20	Real	55.3717
LONGITUDE	22-30	Real	-7.3400
ELEVATION	32-37	Real	21.0
STATE	39-40	Character
NAME	42-71	Character	MALIN HEAD
GSN FLAG	73-75	Character	GSN
HCN/CRN FLAG	77-79	Character
WMO ID	81-85	Character	03980
```

A sample query:
```
AE000041196  25.3330   55.5170   34.0    SHARJAH INTER. AIRP            GSN     41196
```

In [9]:
stations = pd.read_fwf('s3://noaa-ghcn-pds/ghcnd-stations.txt', widths=[12, 9, 10, 7, 3, 31, 4, 4, 6], header=None)
stations.columns = ["ID", "LATITUDE", "LONGITUDE", "ELEVATION", "STATE", "NAME", "GSN FLAG", "HCN/CRN FLAG", "WMO ID"]
stations

,ID,LATITUDE,LONGITUDE,ELEVATION,STATE,NAME,GSN FLAG,HCN/CRN FLAG,WMO ID
0,ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,NaN
1,ACW00011647,17.1333,-61.7833,19.2,NaN,ST JOHNS,NaN,NaN,NaN
2,AE000041196,25.3330,55.5170,34.0,NaN,SHARJAH INTER. AIRP,GSN,NaN,41196.0
3,AEM00041194,25.2550,55.3640,10.4,NaN,DUBAI INTL,NaN,NaN,41194.0
4,AEM00041217,24.4330,54.6510,26.8,NaN,ABU DHABI INTL,NaN,NaN,41217.0
...,...,...,...,...,...,...,...,...,...
118487,ZI000067969,-21.0500,29.3670,861.0,NaN,WEST NICHOLSON,NaN,NaN,67969.0
118488,ZI000067975,-20.0670,30.8670,1095.0,NaN,MASVINGO,NaN,NaN,67975.0
118489,ZI000067977,-21.0170,31.5830,430.0,NaN,BUFFALO RANGE,NaN,NaN,67977.0
118490,ZI000067983,-20.2000,32.6160,1132.0,NaN,CHIPINGE,GSN,NaN,67983.0


In [10]:
country_codes = pd.read_csv("s3://noaa-ghcn-pds/ghcnd-countries.txt", delimiter="\n", header=None)[0].str.extract('(?P<code>.{2})(?P<country>.{0,})')
country_codes

,code,country
0,AC,Antigua and Barbuda
1,AE,United Arab Emirates
2,AF,Afghanistan
3,AG,Algeria
4,AJ,Azerbaijan
...,...,...
214,WI,Western Sahara
215,WQ,Wake Island [United States]
216,WZ,Swaziland
217,ZA,Zambia


In [11]:
# Global variables
elements = ["PRCP", "SNOW", "SNWD", "TMAX", "TMIN"]
all_elements = list(inventory["ELEMENT"].unique())
years = list(range(1763, 2022))

## Utility Functions
Some helpful functions in helping with EDA and modeling

In [12]:
def df_loader(year, local=False):
    if local:
        df = pd.read_csv('data/' + str(year) + '.csv', header=None)
    else:
        df = pd.read_csv('s3://noaa-ghcn-pds/csv/' + str(year) + '.csv', header=None)
    df.columns = ["ID", "YEAR/MONTH/DAY", "ELEMENT", "DATA VALUE", "M-FLAG", "Q-FLAG", "S-FLAG", "OBS-TIME"]
    df["YEAR/MONTH/DAY"] = pd.to_datetime(df["YEAR/MONTH/DAY"], format="%Y%m%d")
    return df

In [13]:
def df_filter(df, _id, element):
    return df.loc[(df["ID"] == _id) & (df["ELEMENT"] == element)][["YEAR/MONTH/DAY", "DATA VALUE"]].set_index("YEAR/MONTH/DAY").sort_index()

In [14]:
#same as df_filter, but a vector of ALL the data
def df_filter_vector(_id, element, local=False, custom_years=None):
    df_vector = pd.DataFrame(columns=["DATA VALUE"])
    if custom_years:
        years = range(custom_years[0], custom_years[1] + 1)
    for year in tqdm(years):
        if local:
            df = df_filter(dfs[year], _id, element)
        else:
            df = df_filter(df_loader(year), _id, element)
            
        if df_vector.empty:
            df_vector = df
        else:
            df_vector = df_vector.append(df)
    return df_vector

In [15]:
#print statements are replaced with tqdm functions
# MatPlotLib runs on single thread, we can parallelize plotting by using Ray
@ray.remote
def plotter(df, _id, element, save=False, year=None):
    station_name = stations.loc[stations['ID'] == _id]["NAME"].item()
    df = df_filter(df, _id, element)
    
    if df.empty:
        tqdm.write(element + " data on " + str(year) + " for " + station_name + " with id: " + _id + " is empty. Plotting is skipped.")
        return

    plt.figure(figsize=(20, 10))
    plt.plot(df)
    plt.title(element + " Data at " + station_name)
    plt.xlabel("Date")
    plt.ylabel("Value")
    #plt.xlim() #figure out a way to set limit of year
    plt.ylim(-100, 400)
    if save:
        directory = "figures/" + _id + "/" + element
        try:
            # Create target directory
            os.makedirs(directory)
            tqdm.write(f"Directory " + directory + " Created ")
        except FileExistsError:
            tqdm.write(f"Directory " + directory + " already exists. File " + directory + "/" + str(year) + ".png has been saved successfully.")
        
        plt.savefig(directory + "/" + str(year) + ".png")
    plt.clf()

In [16]:
def rmse(actual, expected):
    """
    Computes the root mean squared error to evaluate models/predictions. It can accept all the datatypes used
    in this notebook
    """
    if type(actual) != type(expected):
        raise TypeError("actual and expected must be the same types")
    if type(actual) == np.ndarray:
        return np.sqrt(np.mean((expected - actual) ** 2))
    elif type(actual) == nums.core.array.blockarray.BlockArray:
        return nps.sqrt(nps.mean((expected - actual) ** 2))
    elif type(actual) == modin.pandas.dataframe.DataFrame or type(actual) == pandas.core.frame.DataFrame:
        #TODO
        raise NotImplementedError
    else:
        raise TypeError

In [17]:
def design_matrix(years, elements, target=None, convert_nps=False, local=False):
    """
    Set target to your "y" predictor. If y has NaNs or missing values, we will drop the data row.
    """
    df_design = pd.DataFrame()
    
    for year in tqdm(years):
        if local:
            df = dfs[year]
        else:
            df = df_loader(year)
            

        if target[0] not in df["ELEMENT"].unique():
            continue

        df = df[df['ELEMENT'].isin(elements)]
        df = pd.pivot_table(df, index=["ID", "YEAR/MONTH/DAY"], columns="ELEMENT", values="DATA VALUE").reset_index(level=[0,1])
        df = df.merge(stations[["ID", "LATITUDE", "LONGITUDE", "ELEVATION"]], how='inner', on='ID')
        
        if target:
            df = df.dropna(subset=target)
        df = df.dropna() #TODO add a flag to toggle this
        
        
        df["YEAR/MONTH/DAY"] = df["YEAR/MONTH/DAY"].apply(lambda x: pd.Period(x, freq='D').day_of_year)
        df["TMAX"] = df["TMAX"] / 10
        df["TMIN"] = df["TMIN"] / 10
        df["AVG"] = (df["TMAX"] + df["TMIN"]) / 2
        df["RANGE"] = df["TMAX"] - df["TMIN"]
        #df = df.drop(["ID"], axis=1)
        #df = df.astype(float)
        
        if df_design.empty:
            df_design = df
        else:
            df_design = df_design.append(df)
        
        
    
    if convert_nps:
        return nps.array(result.to_numpy().astype(np.double))
    return df_design

In [18]:
def design_matrix_time_series_stack(_id, element, years, convert_nps=True, local=False):
    """
    Inputs are station ID and element
    Output is a design matrix of time series per year stacked on top of each other.
    rows are year of data collected, 
    cols are day of the year
    
    returns NumS array or Pandas DataFrame
    """
    df_design = pd.DataFrame(columns=pd.date_range(start="2020-01-01", end="2020-12-31").strftime('%m-%d'))
    station_name = stations.loc[stations['ID'] == _id]["NAME"].item()
    
    for year in tqdm(years):
        if local:
            df = dfs[year]
        else:
            try:
                df = df_loader(year)
            except ClientError:
                tqdm.write(str(year) + ".csv doesn't exist on remote, addition to design matrix is skipped.")
                continue
            
        df = df_filter(df, _id, element)
        if df.empty:
            tqdm.write(element + " data on " + str(year) + " for " + station_name + " with id: " + _id + " is empty. Addition to design matrix is skipped.")
            continue
        df.index = df.index.strftime('%m-%d')
        df.columns = [year]
        df = df.T
        df_design = df_design.append(df)


    df_design.index.name = None
    if convert_nps:
        #return nums_modin.from_modin(df_design) #uncomment this once bug gets fixed. 
        return nps.array(df_design.to_numpy().astype(np.double))
    return df_design

## Loading all the DataFrames to Memory
We can also store all the dataframes into memory, given that there is enough RAM. Dataset is ~100GB, so something more that that should be fine. (Uses 200GB+ of memory on this system)

Skip this cell if running on less RAM.

In [ ]:
%%time
dfs = {}
for year in tqdm(years):
    dfs[year] = df_loader(year, local=local) 

  0%|          | 0/259 [00:00<?, ?it/s]

Loading all the dataframes from local storage to memory gives us these times:

**modin**:
```
CPU times: user 2min 48s, sys: 54 s, total: 3min 42s
Wall time: 5min 14s
```

**pandas**
```
CPU times: user 24min 22s, sys: 5min 39s, total: 30min 2s
Wall time: 29min 32s
```

As we can see, Modin gives us ~6x speedup, which is impressive considering that ~100GB is being transferred and loaded onto memory.

# EDA
Visualizations and understanding the data. We can plot how many data points each country has collected in their weather stations.

In [ ]:
station_data_freq = pd.DataFrame()
years.reverse() #reverse, so NaNs can be filled in to missing/old data

for year in tqdm(years):
    station_data_freq[year] = dfs[year].groupby(dfs[year]["ID"].str.slice(stop=2))["DATA VALUE"].sum()

years.reverse()
station_data_freq = station_data_freq.fillna(0).T
station_data_freq

In [ ]:
for index, row in tqdm(country_codes.iterrows()): 
    if row[0] in station_data_freq.T.columns:
        fig = plt.figure(figsize=(10, 10))
        ax = station_data_freq.T[row[0]].plot();
        ax.set_xlabel("Years")
        ax.set_ylabel("Number of Data Points")
        plt.savefig(("figures/datasamples/" + row[1] + ".png").replace(" ", "_"))
        plt.clf()

For example, this is number of data points for United States station.

![](figures/datasamples/_United_States_.png)

In [ ]:
sample_points = []
for year in tqdm(years):
    sample_points.append(len(dfs[year].index))

Note that there is a dip at the end. That is due to this dataset being collected and updated live. Thus, it includes the year 2021 (the current year making this notebook).

In [ ]:
plt.figure(figsize=(9, 3))
plt.plot(years, sample_points);
plt.title("Number of Data Points Collected Per Year")
plt.ticklabel_format(style='plain')
plt.xlabel("Year")
plt.ylabel("Number of Data Points");

In [ ]:
# Massively plots elements per year for each id
"""
for _id in tqdm(stations["ID"]):
    for element in elements:
        for year in years:
            plotter.remote(dfs[year], _id, element, save=True, year=year)
"""

We can also analyze what type of weather data is being collected. Here is a plot of the top 25 weather data being collected ranked by frequency.

In [ ]:
inventory["ELEMENT"].value_counts().head(25).plot(kind="bar");
plt.title("Top Weather Data Type Collected")
plt.xlabel("Type")
plt.ylabel("Count")

We can also analyze location of each station by plotting their longitude and latitude to give us a familiar outline:

In [ ]:
plt.figure(figsize=(20, 10))
plt.title("Scatterplot of Weather Stations")
plt.xlabel("Longitude")
plt.xlabel("Latitude")
plt.scatter(x=stations['LONGITUDE'], y=stations['LATITUDE'], s=0.9, alpha=0.7);

Using plotly, we can make this more interactive and visualize the density of these weather stations. Note that for some libraries that need to use Pandas, we can insert `._to_pandas()` to revert a modin dataframe to pandas. First we can plot a density heatmap to show where each station is located. It also shows us that there are more weather stations in densly populated areas.

Source/motivation: https://medium.com/plotly/how-to-create-2d-and-3d-interactive-weather-maps-in-python-and-r-77ddd53cca8

Since plotly plots can take up space, it won't be uploaded to the repo. But after running, it can be viewed through this link after it's saved to `.html` file.

Link: http://127.0.0.1:8888/files/station_density.html

(If link doesn't work, make sure port number is same. It is set to the jupyter default port 8888)

In [ ]:
fig = px.density_mapbox(stations._to_pandas(), lat='LATITUDE', lon='LONGITUDE', radius=5,
                        center=dict(lat=0, lon=180), zoom=0,
                        mapbox_style="stamen-terrain")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.write_html("station_density.html")

In [ ]:
df = design_matrix([2010], ["TMAX", "TMIN"], target=["TMAX"], local=True)
df["TMAX"] = df["TMAX"] / 10
df["TMIN"] = df["TMIN"] / 10
df

We can also plot the TMAX density of temperature.

Link: http://127.0.0.1:8888/files/tmax_density.html

In [ ]:
import plotly.graph_objects as go
temp = df[df["YEAR/MONTH/DAY"] == 1]._to_pandas()
fig = go.Figure(go.Densitymapbox(lat=temp.LATITUDE, lon=temp.LONGITUDE, z=temp.TMAX,
                                 radius=1))
fig.update_layout(mapbox_style="stamen-terrain", mapbox_center_lon=180)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.write_html("tmax_density.html")

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()

# Add traces, one for each slider step
for day in tqdm(np.arange(1, 367, 1)):
    temp = df[df["YEAR/MONTH/DAY"] == day]._to_pandas()
    fig.add_trace(
        go.Densitymapbox(lat=temp.LATITUDE, lon=temp.LONGITUDE, z=temp.TMAX,
                                 radius=10, visible=False, 
                        name="Temperature Max on Day "+ str(day))
    )

# Make 10th trace visible
fig.data[0].visible = True

# Create and add slider
steps = []

for i in tqdm(range(len(fig.data))):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Slider switched to Day: " + str(i)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={"prefix": "Frequency: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders,
    margin={"r":0,"t":0,"l":0,"b":0},
    mapbox_style="stamen-terrain", 
    mapbox_center_lon=180
)

fig.write_html("tmax_density_timelapse.html")

Next, let's analyze and plot data from a weather station. We will analyze the station USC00040693 in Berkeley, CA [(Google Maps)](https://www.google.com/maps/place/37°52'27.8%22N+122°15'38.2%22W/@37.8744024,-122.2618614,17z/data=!3m1!4b1!4m5!3m4!1s0x0:0x0!8m2!3d37.8744!4d-122.2606). Through Pandas/modin, we can figure out which elements of weather data this station records:

In [ ]:
inventory[inventory["ID"] == 'USC00040693']

Data will vary a lot between stations, some stations may record as little as just precipitation. Coincidentally, this weather station is located on UC Berkeley campus and has been recording data of various elements for quite a while. The more interesting data values recorded in this station other than temperature and precipitation are the WT** data types. According to the [documentation](https://docs.opendata.aws/noaa-ghcn-pds/readme.html) they are mapped to:

* **01 = Fog, ice fog, or freezing fog (may include heavy fog)**
* 02 = Heavy fog or heaving freezing fog (not always distinguished from fog)
* **03 = Thunder**
* **04 = Ice pellets, sleet, snow pellets, or small hail**
* **05 = Hail (may include small hail)**
* 06 = Glaze or rime
* 07 = Dust, volcanic ash, blowing dust, blowing sand, or blowing obstruction
* **08 = Smoke or haze**
* 09 = Blowing or drifting snow
* 10 = Tornado, waterspout, or funnel cloud
* **11 = High or damaging winds**
* 12 = Blowing spray
* 13 = Mist
* **14 = Drizzle**
* 15 = Freezing drizzl
* 16 = **Rain (may include freezing rain, drizzle, and freezing drizzle)**
* 17 = Freezing rain
* 18 = Snow, snow pellets, snow grains, or ice crystals
* 19 = Unknown source of precipitation
* 21 = Ground fog
* 22 = Ice fog or freezing fog

Unfortunately, plotting these special elements out shows us there is nothing interesting going.

In [ ]:
#TODO: Make dataloading faster by batching operations into a design matrix with features, if possible. 
#Currently 7-8s per iteration, at most 12 min per vector. See if this improves when scaled.
berkeley_weather_elements = inventory[inventory["ID"] == 'USC00040693'][["ELEMENT", "FIRSTYEAR", "LASTYEAR"]]
berkeley_weather_elements = berkeley_weather_elements[berkeley_weather_elements["ELEMENT"].isin(["TMAX", "TMIN", "PRCP"])] #override to save time
berkeley_time_series = {}

for _, rows in berkeley_weather_elements.iterrows():
    element, firstyear, lastyear = rows
    berkeley_time_series[element] = df_filter_vector('USC00040693', element, local=local, custom_years=(lastyear-20, lastyear - 1)) #only grabbing the last 20 years for effiency

We can plot data in the last 3 years, (plotting all of the data will take a long time due to the bottleneck of matplotlib).

In [ ]:
berkeley_time_series["TMAX"].tail(365 * 3).plot();

In [ ]:
berkeley_time_series["TMIN"].tail(365 * 3).plot();

In [ ]:
berkeley_time_series["PRCP"].tail(365 * 3).plot();

# Modeling
#TODO: Refactor modeling portion to a separate notebook after done experimenting

## Simple Modeling with FFT
Using NumPy's FFT module to forecast seasonal weather patterns by extrapolation. A function will be extrapolated with FFT on the training dataset and then we can evaluate how well we extrapolated using root mean squared error. We can also tune the variables such as `ratio`, `predict`, and `n_harm` which represents ratio between training and test set, number of days to predict in test set, and frequency of FFT respectively. Tuning these will help avoiding overfitting.

* Source: https://gist.github.com/tartakynov/83f3cd8f44208a1856ce
* StackOverflow Post: https://stackoverflow.com/questions/4479463/using-fourier-analysis-for-time-series-prediction

In [ ]:
def fourier_extrapolation(X, n_predict, n_harm=100):
    n = X.size
    t = np.arange(0, n)
    p = np.polyfit(t, X, 1)            # find linear trend in x
    X_notrend = X - p[0] * t           # detrended x
    X_freqdom = np.fft.fft(X_notrend)  # detrended x in frequency domain
    f = np.fft.fftfreq(n)              # frequencies
    indexes = list(range(n))
    indexes.sort(key = lambda i: np.absolute(f[i]))
 
    t = np.arange(0, n + n_predict)
    restored_sig = np.zeros(t.size)
    for i in indexes[:1 + n_harm * 2]:
        ampli = np.absolute(X_freqdom[i]) / n   # amplitude
        phase = np.angle(X_freqdom[i])          # phase
        restored_sig += ampli * np.cos(2 * np.pi * f[i] * t + phase)
    return restored_sig + p[0] * t

In [ ]:
ratio = 6      # Ratio will be train:test = ratio:1
predict = 366 * 3 # Number of future days to predict
n_harm = 15

X_train = berkeley_time_series["TMIN"].sort_index().iloc[-ratio * predict:-predict].to_numpy().reshape(-1) 
X_test = berkeley_time_series["TMIN"].sort_index().iloc[-predict:].to_numpy().reshape(-1) 

extrapolation = fourier_extrapolation(X_train, predict, n_harm=n_harm)
plt.figure(figsize=(50, 10))
plt.plot(np.arange(0, extrapolation.size), extrapolation, 'r', label = 'extrapolation', linewidth=3)
plt.plot(np.arange(0, X_train.size), X_train, 'b', label = r'training data', linewidth = 3)
plt.plot(np.arange(X_train.size, extrapolation.size), X_test, 'g', label = r'test data')
plt.legend()
plt.savefig("fft_berkeley_tmin.png")

In [ ]:
print("Training RMSE:", rmse(extrapolation[:X_train.size], X_train))
print("Test RMSE:", rmse(extrapolation[-X_test.size:], X_test))

For more sporadic data like precipitation, it's much more harder to predict and identify trends with FFT, thus giving a large RMSE value compared to temperature.

In [ ]:
ratio = 4
predict = 366 * 2 # Number of future days to predict
n_harm = 500

X_train = berkeley_time_series["PRCP"].sort_index().iloc[-ratio * predict:-predict].to_numpy().reshape(-1) 
X_test = berkeley_time_series["PRCP"].sort_index().iloc[-predict:].to_numpy().reshape(-1) 

extrapolation = fourier_extrapolation(X_train, predict, n_harm=n_harm)
plt.figure(figsize=(50, 10))
plt.plot(np.arange(0, extrapolation.size), extrapolation, 'r', label = 'extrapolation', linewidth=3)
plt.plot(np.arange(0, X_train.size), X_train, 'b', label = r'training data', linewidth = 3)
plt.plot(np.arange(X_train.size, extrapolation.size), X_test, 'g', label = r'test data')
plt.legend()
plt.savefig("fft_berkeley_prcp.png")

In [ ]:
print("Training RMSE:", rmse(extrapolation[:X_train.size], X_train))
print("Test RMSE:", rmse(extrapolation[-X_test.size:], X_test))

## Linear Regression with NumS and Time Series Data

In [ ]:
from nums.models.glms import LinearRegression

In [ ]:
y_train = berkeley_time_series["TMIN"].iloc[-4000:-1000].to_numpy().reshape(-1) 
y_test = berkeley_time_series["TMIN"].iloc[-1000:].to_numpy().reshape(-1) 

#convert to NumS arrays, assigning to y since we want X to be time, y to be TMIN (data value)
y_train = nps.array(y_train)
y_test = nps.array(y_test)

In [ ]:
X_train = nps.arange(0, y_train.shape[0], 1).reshape(-1, 1).astype(np.double)
X_test = nps.arange(y_train.shape[0], y_train.shape[0] + y_test.shape[0], 1).reshape(-1, 1).astype(np.double)

In [ ]:
plt.plot(X_train.get(), y_train.get())

We can attempt to fit the time series vector of temperature with NumS linear regression. Nothing interesting should be happening, but shows us how NumS can solve linear regression in a simple case.

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

m = model._beta.get()
b = model._beta0.get()

plt.plot(X_train.get(), y_train.get())
plt.plot(X_train.get(), m * X_train.get() + b)

Model also reveals to use that there is only a slight positive trend in data with the slope obtained from:

In [ ]:
model._beta.get()

## Creating the Design Matrix for Predicting/Classifying Precipitation
Next, we can create a design matrix with multiple features as elements and perform classification. In the next few cells, we will try to classify whether or not snow will fall from data all around the world! We will use several features to help us determine if there is precipitation or not such as temperature, location, elevation, and day of the year. Note that what we are doing is *time-independent* opposed to the previous example with time series. We will explore various popular models in classical machine learning.

In [ ]:
%%time
test_elements = ["PRCP", "TMAX", "TMIN"]
data = design_matrix(years[-10:], test_elements, target=["PRCP"], convert_nps=False, local=local)
data = data[data["PRCP"] >= 0]
data

Going back to our EDA, we can visualize the design matrix through plotly by plotting the precipitaton during a certain day of the year. Here, we can plot the precipication on January 1st 2020. We observe that there is a lot of rain happening at higher elevations (mountains) in the US. There are also data at other places, but recalling our previous station density map, it's not as dense in other places.

http://127.0.0.1:8888/files/prcp_dmatrix_density.html

In [ ]:

plot = design_matrix([2020], test_elements, target=["PRCP"], convert_nps=False, local=local)
plot = plot[plot["PRCP"] >= 0]
plot = plot[(plot["YEAR/MONTH/DAY"] == 1) & (plot["PRCP"] > 0)]
fig = go.Figure(go.Densitymapbox(lat=plot.LATITUDE, lon=plot.LONGITUDE, z=plot.PRCP,
                                 radius=10))
fig.update_layout(mapbox_style="stamen-terrain", mapbox_center_lon=180)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.write_html("prcp_dmatrix_density.html")

Here, we will perform a train-test-split based on station IDs.

In [ ]:
ids = np.random.permutation(data["ID"].unique())
split = int(ids.shape[0] * .8)
train_ids = ids[:split]
test_ids = ids[split:]

In [ ]:
train = data[data["ID"].isin(train_ids)]
test = data[data["ID"].isin(test_ids)]
#'TMAX', 'TMIN','AVG', 'RANGE',
X_train = train[['YEAR/MONTH/DAY', 'TMAX', 'TMIN','AVG', 'RANGE', 'LATITUDE', 'LONGITUDE', 'ELEVATION']].to_numpy().astype(np.double)
y_train = train['PRCP'].to_numpy().astype(np.double)
X_test = test[['YEAR/MONTH/DAY', 'TMAX', 'TMIN','AVG', 'RANGE', 'LATITUDE', 'LONGITUDE', 'ELEVATION']].to_numpy().astype(np.double)
y_test = test['PRCP'].to_numpy().astype(np.double)

# NumS arrays must maintain block shapes, code below ensures they are shaped correctly.
X_train = nps.array(X_train)
y_train = nps.array(y_train)
X_test = nps.array(X_test)
y_test = nps.array(y_test)
y_train = y_train.reshape(block_shape=(X_train.block_shape[0],))
y_test = y_test.reshape(block_shape=(X_test.block_shape[0],))

## Linear Regression with NumS and Multiple Features
Next , we'll use linear regression with NumS to see if we can classify precipitation.

In [ ]:
from nums.models.glms import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
training_results = model.predict(X_train).get()
test_results = model.predict(X_test).get()

print("Training RMSE:", rmse(training_results, y_train.get()))
print("Testing RMSE", rmse(test_results, y_test.get()))

## Logistic Regression with NumS and Multiple Features
A continutation from the previous module, we can also use NumS logistic regression to classify if it will precipitation or not.

We can set y to be a binary variable, but we need to do some hacks since advanced indexing is not supported yet on NumS

In [ ]:
# convert y_train and y_test to binary variable
y_train_block_shape = y_train.block_shape
y_train = y_train.get()
y_train[y_train > 0] = 1.0
y_train = nps.array(y_train)
y_train = y_train.reshape(block_shape=y_train_block_shape)


y_test_block_shape = y_test.block_shape
y_test = y_test.get()
y_test[y_test > 0] = 1.0
y_test = nps.array(y_test)
y_test = y_test.reshape(block_shape=y_test_block_shape)

In [ ]:
from nums.models.glms import LogisticRegression

# since matrix could be singular, a mask is added on to X_train to help invert it
mask = nps.random.rand(X_train.shape[0], X_train.shape[1]) * 0.00001

model = LogisticRegression()
model.fit(X_train + mask, y_train)

In [ ]:
training_results = model.predict(X_train).get()
test_results = model.predict(X_test).get()

print("Training Accuracy:", np.sum(training_results == y_train.get()) / y_train.shape[0])
print("Test Accuracy:", np.sum(test_results == y_test.get()) / y_test.shape[0])

## Decision Trees with Scikitlearn
We can also use Scikit-learn's Decision Tree to train and predict if there is precipitation or not. We can also use that model to plot out the features it splits on, giving us more insight on what the motivating features are. For the purposes of plotting, the depth is limited to 3.

In [ ]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth=3)
model.fit(X_train.get(), y_train.get())

In [ ]:
print("Training Accuracy:", model.score(X_train.get(), y_train.get()))
print("Test Accuracy:", model.score(X_test.get(), y_test.get()))

In [ ]:
columns = ['YEAR/MONTH/DAY', 'TMAX', 'TMIN','AVG', 'RANGE', 'LATITUDE', 'LONGITUDE', 'ELEVATION']
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10,10), dpi=500);

tree.plot_tree(model, feature_names=columns, class_names=["NO PRCP", "PRCP"], filled=True);

fig.savefig('decision_tree_prcp.png', dpi=500, transparent=False)

## XGBoost with Modin (Experimental)
We can also perform classification with XGBoost within the expermeintal package in Modin.

* Documentation: https://modin.readthedocs.io/en/latest/modin_xgboost.html

In [ ]:
import modin.experimental.xgboost as xgb

In [ ]:
dtrain = xgb.DMatrix(pd.DataFrame(X_train.get()), pd.DataFrame(y_train.get()))
dtest = xgb.DMatrix(pd.DataFrame(X_test.get()), pd.DataFrame(y_test.get()))

In [ ]:
xgb_params = {
    "eta": 0.3,
    "max_depth": 3,
    "objective": "multi:softprob",
    "num_class": 2,
    "eval_metric": "mlogloss",
}
steps = 20

# Create dict for evaluation results
evals_result = dict()

# Run training
model = xgb.train(
    xgb_params,
    dtrain,
    steps,
    evals=[(dtrain, "train")],
    evals_result=evals_result
)

# Print evaluation results
print(f'Evals results:\n{evals_result}')

# Predict results
prediction = model.predict(dtest)

# Print prediction results
print(f'Prediction results:\n{prediction}')

In [ ]:
y_result = prediction[1].to_numpy()
y_result[y_result >= 0.5] = 1
y_result[y_result < 0.5] = 0
np.sum(y_result == y_test.get()) / y_result.shape[0]

## Random Forests with Ray Backend
We can also perform random forest classifier with Ray by replacing the parallel backend joblib with Ray. This will automatically distribute Decision Trees across multiple threads/clusters.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import joblib
from ray.util.joblib import register_ray
register_ray()

with joblib.parallel_backend('ray'):
    model = RandomForestClassifier(n_jobs=n_jobs) #TODO, change n_jobs to match the number of threads in cluster
model.fit(X_train.get(), y_train.get())

In [ ]:
print("Training Accuracy:", model.score(X_train.get(), y_train.get()))
print("Test Accuracy:", model.score(X_test.get(), y_test.get()))

## Modeling (Again but with Classifying Snow)
Next, we will repeat the same models but with snow. But as we see when we graph it, we are limited to only US and Canada data, as other countries don't support snow data.

In [ ]:
%%time
test_elements = ["SNOW", "TMAX", "TMIN"]
data = design_matrix(years[-10:], test_elements, target=["SNOW"], convert_nps=False, local=local)
data = data[data["SNOW"] >= 0]
data

As mentioned before, we can plot the data, and graph it onto a map to see that Snow data is only available on US and Canada

http://127.0.0.1:8888/files/snow_dmatrix_density.html

And for comparison, we see that not all precipation data is snow. An interesting observation is that sometimes snow in mountain happens at higher elevations:

http://127.0.0.1:8888/files/prcp_dmatrix_density.html

In [ ]:
plot = design_matrix([2020], test_elements, target=["SNOW"], convert_nps=False, local=local)
plot = plot[plot["SNOW"] >= 0]
plot = plot[(plot["YEAR/MONTH/DAY"] == 1) & (plot["SNOW"] > 0)]
fig = go.Figure(go.Densitymapbox(lat=plot.LATITUDE, lon=plot.LONGITUDE, z=plot.SNOW,
                                 radius=10))
fig.update_layout(mapbox_style="stamen-terrain", mapbox_center_lon=180)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.write_html("snow_dmatrix_density.html")

In [ ]:
ids = np.random.permutation(data["ID"].unique())
split = int(ids.shape[0] * .8)
train_ids = ids[:split]
test_ids = ids[split:]

In [ ]:
train = data[data["ID"].isin(train_ids)]
test = data[data["ID"].isin(test_ids)]
#'TMAX', 'TMIN','AVG', 'RANGE',
X_train = train[['YEAR/MONTH/DAY', 'TMAX', 'TMIN','AVG', 'RANGE', 'LATITUDE', 'LONGITUDE', 'ELEVATION']].to_numpy().astype(np.double)
y_train = train['SNOW'].to_numpy().astype(np.double)
X_test = test[['YEAR/MONTH/DAY', 'TMAX', 'TMIN','AVG', 'RANGE', 'LATITUDE', 'LONGITUDE', 'ELEVATION']].to_numpy().astype(np.double)
y_test = test['SNOW'].to_numpy().astype(np.double)

# NumS arrays must maintain block shapes, code below ensures they are shaped correctly.
X_train = nps.array(X_train)
y_train = nps.array(y_train)
X_test = nps.array(X_test)
y_test = nps.array(y_test)
y_train = y_train.reshape(block_shape=(X_train.block_shape[0],))
y_test = y_test.reshape(block_shape=(X_test.block_shape[0],))

## Linear Regression with Multiple Features (Snow)

In [ ]:
from nums.models.glms import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
training_results = model.predict(X_train).get()
test_results = model.predict(X_test).get()

print("Training RMSE:", rmse(training_results, y_train.get()))
print("Testing RMSE", rmse(test_results, y_test.get()))

## Logistic Regression with Multiple Features (Snow)

In [ ]:
# convert y_train and y_test to binary variable
y_train_block_shape = y_train.block_shape
y_train = y_train.get()
y_train[y_train > 0] = 1.0
y_train = nps.array(y_train)
y_train = y_train.reshape(block_shape=y_train_block_shape)


y_test_block_shape = y_test.block_shape
y_test = y_test.get()
y_test[y_test > 0] = 1.0
y_test = nps.array(y_test)
y_test = y_test.reshape(block_shape=y_test_block_shape)

In [ ]:
from nums.models.glms import LogisticRegression

# since matrix could be singular, a mask is added on to X_train to help invert it
mask = nps.random.rand(X_train.shape[0], X_train.shape[1]) * 0.00001

model = LogisticRegression()
model.fit(X_train + mask, y_train)

In [ ]:
training_results = model.predict(X_train).get()
test_results = model.predict(X_test).get()

print("Training Accuracy:", np.sum(training_results == y_train.get()) / y_train.shape[0])
print("Test Accuracy:", np.sum(test_results == y_test.get()) / y_test.shape[0])

## Decision Trees with Scikitlearn (Snow)

In [ ]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth=4)
model.fit(X_train.get(), y_train.get())

In [ ]:
print("Training Accuracy:", model.score(X_train.get(), y_train.get()))
print("Test Accuracy:", model.score(X_test.get(), y_test.get()))

In [ ]:
columns = ['YEAR/MONTH/DAY', 'TMAX', 'TMIN','AVG', 'RANGE', 'LATITUDE', 'LONGITUDE', 'ELEVATION']
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10,10), dpi=500);

tree.plot_tree(model, feature_names=columns, class_names=["NO SNOW", "SNOW"], filled=True);

fig.savefig('decision_tree_snow.png', dpi=500, transparent=False)

## XGBoost with Modin (Snow)

In [ ]:
import modin.experimental.xgboost as xgb

In [ ]:
dtrain = xgb.DMatrix(pd.DataFrame(X_train.get()), pd.DataFrame(y_train.get()))
dtest = xgb.DMatrix(pd.DataFrame(X_test.get()), pd.DataFrame(y_test.get()))

In [ ]:
xgb_params = {
    "eta": 0.3,
    "max_depth": 3,
    "objective": "multi:softprob",
    "num_class": 2,
    "eval_metric": "mlogloss",
}
steps = 20

# Create dict for evaluation results
evals_result = dict()

# Run training
model = xgb.train(
    xgb_params,
    dtrain,
    steps,
    evals=[(dtrain, "train")],
    evals_result=evals_result
)

# Print evaluation results
print(f'Evals results:\n{evals_result}')

# Predict results
prediction = model.predict(dtest)

# Print prediction results
print(f'Prediction results:\n{prediction}')

In [ ]:
y_result = prediction[1].to_numpy()
y_result[y_result >= 0.5] = 1
y_result[y_result < 0.5] = 0
np.sum(y_result == y_test.get()) / y_result.shape[0]

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import joblib
from ray.util.joblib import register_ray
register_ray()

with joblib.parallel_backend('ray'):
    model = RandomForestClassifier(n_jobs=n_jobs) #TODO, change n_jobs to match the number of threads in cluster
model.fit(X_train.get(), y_train.get())

In [ ]:
print("Training Accuracy:", model.score(X_train.get(), y_train.get()))
print("Test Accuracy:", model.score(X_test.get(), y_test.get()))

## Using Models to Learn New Locations
Using a model, can we possible "learn" a new location that doesn't have a weather station? We can create a meshgrid of longitude and latitude, just by location, we will try to clasify snow. Of coures, we have to discard features like temperature. We also have to discard elevation, but if there happens to be detailed terrain data for longitude, latitude, and elevation, it could definitely help. 

In [ ]:
x = np.linspace(-90, 90, 10000)
y = np.linspace(-180, 180, 10000)

geo_data = []

for i in tqdm(range(x.shape[0])):
    for j in range(y.shape[0]):
        geo_data.append((x[i], y[j]))
        
new_location_df = pd.DataFrame(geo_data, columns=["lat", "lon"])
new_location_df["day"] = 1
new_location_df = new_location_df.reindex(columns=['day', 'lat', 'lon'])
location_grid = new_location_df.to_numpy().astype(np.double)

For this, we will only predict snow at unknown locations on Januaury 1st, 2021.

In [ ]:
df = data[["YEAR/MONTH/DAY", "LATITUDE", "LONGITUDE", "SNOW"]]
df = df[df["YEAR/MONTH/DAY"] == 1]
X = df[["YEAR/MONTH/DAY", "LATITUDE", "LONGITUDE"]]
y = df["SNOW"]

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import joblib
from ray.util.joblib import register_ray
register_ray()

with joblib.parallel_backend('ray'):
    model = RandomForestClassifier(n_jobs=n_jobs)
model.fit(X, y)
print("Training Accuracy:", model.score(X, y))

In [ ]:
# Predict new locations
snow = model.predict(location_grid)
new_location_df["snow"] = snow
new_location_df

For plotting purposes, we can just filter out where it snows. It'll be obvious in the plot where we set the boundaries.

In [ ]:
new_location_df = new_location_df[new_location_df["snow"] == 1]
new_location_df

In [ ]:
fig = go.Figure(go.Densitymapbox(lon=new_location_df.lon, lat=new_location_df.lat, radius=5))
fig.update_layout(mapbox_style="stamen-terrain", mapbox_center_lon=180)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.write_html("new_locations_snow.html")

In [ ]:
actual = data[["YEAR/MONTH/DAY", "LATITUDE", "LONGITUDE", "SNOW"]]
actual = actual[(actual["YEAR/MONTH/DAY"] == 1) & (actual["SNOW"] > 0)]
actual

In [ ]:
fig = go.Figure(go.Densitymapbox(lon=actual.LONGITUDE, lat=actual.LATITUDE,
                                 radius=5))
fig.update_layout(mapbox_style="stamen-terrain", mapbox_center_lon=180)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.write_html("actual_snow.html")

## PCA with NumS
We can stack years of data on top of each other to give a design matrix of years as cols/features, and day of the year as rows/data point. Running PCA can tell us if there is any abnormality weather between the years. Continuing with the data from Berkeley, this is what the data shows:

Source/inspriation: https://towardsdatascience.com/the-pca-trick-with-time-series-d40d48c69d28

In [ ]:
berkeley_tmax_design_matrix = design_matrix_time_series_stack('USC00040693', 'TMAX', years, local=local)

In [ ]:
for i in range(berkeley_tmax_design_matrix.shape[0]):
    plt.plot(berkeley_tmax_design_matrix[i].get());

Seems like the data has one significant feature, which should be expected. Also notice a jump near day 50, indicating that this is around February, when leap days occur, causing inconsistencies in data. There are also some abnormalities in the other dimensions, potentially indicating shifts/differences in climate change.

In [ ]:
%%time
U, S, V = np.linalg.svd(np.nan_to_num(berkeley_tmax_design_matrix.T.get()), full_matrices=False)

In [ ]:
for i in range(25):
    plt.plot(U.T[i])

In [ ]:
from nums.core import linalg
from nums.core import application_manager
nps_app_inst = application_manager.instance()

In [ ]:
%%time
U, S, V = linalg.svd(nps_app_inst, nps.nan_to_num(berkeley_tmax_design_matrix.T))
U.touch() #for timing purposes
S.touch()
V.touch()

In [ ]:
plt.plot(U.T[0, :].get())

Scree Plot of the first 10 singular values

In [ ]:
plt.plot(S[:10].get())

Let's repeat the same with precipication data:

In [ ]:
berkeley_prcp_design_matrix = design_matrix_time_series_stack('USC00040693', 'PRCP', years, local=local)

In [ ]:
for i in range(berkeley_prcp_design_matrix.shape[0]):
    plt.plot(berkeley_prcp_design_matrix[i].get());

In [ ]:
U, S, V = linalg.svd(nps_app_inst, nps.nan_to_num(berkeley_prcp_design_matrix.T))

In [ ]:
for i in range(25):
    plt.plot(U.T[i, :].get());

In [ ]:
#scree plot
plt.plot(S[:10].get());

# End Here
When done, run this so you can delete the large dictionary of dataframes and shutdown Ray properly to free all processes.

In [4]:
del dfs

NameError: name 'dfs' is not defined

In [ ]:
ray.shutdown()